In [174]:
import os
import numpy as np
from ete3 import Tree

class TreeEncoder:

    def encode_tree(self, tree_str):
        """
        Encode the tree structure into a format suitable for input into the neural network.
        """
        # Check if the tree is already encoded
        csv_file = tree_str[:-4] + '.csv'
        if os.path.exists(csv_file):
            return csv_file
        # Call the external script to get CDV encoding
        cmd = f"python -m CDV_full_tree -t {tree_str} > {tree_str[:-4]}.csv"
        os.system(cmd)

    def encode_all_trees(self, trees_directory):
        """
        Encode all the trees in the given directory.
        """
        tree_files = [os.path.join(trees_directory, file) for file in os.listdir(trees_directory) if file.endswith('.nwk')]
        for tree_file in tree_files:
            print(tree_file)
            self.encode_tree(tree_file)


In [181]:
trees_directory = "trees/"
encoder = TreeEncoder()
encoded_trees = encoder.encode_all_trees(trees_directory)

trees/bd.nwk
trees/bisse.nwk
trees/bisseness.nwk
trees/classe.nwk
trees/geosse.nwk
trees/musse.nwk


In [209]:
import pandas as pd

# Load parameter values as a dataframe
tree_files = [os.path.join(trees_directory, file) for file in os.listdir(trees_directory) if file.endswith('.nwk')]

def process_params_musse(param_file):
    # Reads the parameter file and saves the values in a csv file, one tree per row.
    '''
    Example:
    num_states 3
    lambda1  0.581637403143104
    lambda2  0.639469627428334
    lambda3  0.694642654564232
    mu1  0.076379950507544
    mu2  0.339707092673052
    mu3  0.246268530318048
    q12  0.191004574298859
    q13  0.143894099444151
    q21  0.154783235490322
    q23  0.62774416487664
    q31  0.460761926881969
    q32  0.606042030081153
    num_states 3
    ....
    '''
    # If the parameter file is already in the csv format, return
    if param_file.endswith('.csv'):
        return
    with open(param_file, 'r') as f:
        lines = f.readlines()
    param_values = []
    for line in lines:
        if line.startswith('num_states'):
            param_values.append([])
        else:
            param_values[-1].append(line.split()[1])
    param_df = pd.DataFrame(param_values)
    param_df.to_csv(param_file + '.csv', sep='\t', header=False, index=False)
        
        
    

# Create a dataframe to store the parameter values

process_params_musse(param_file=trees_directory + 'musse.params')

cutoff = 250  # Number of trees to process

param_train = pd.read_csv(trees_directory + 'musse.params.csv', nrows=cutoff, header=None, sep='\t', index_col=0)
param_test = pd.read_csv(trees_directory + 'musse.params.csv', sep='\t', header=None, index_col=0)

print(param_train.shape)
print(param_test.shape)

column_names = ['lambda1', 'lambda2', 'lambda3', 'mu1', 'mu2', 'mu3', 'q12', 'q13', 'q21', 'q23', 'q31', 'q32']

def rename_columns(df, names):
    df = df.rename(columns={i: names[int(i)-1] for i in df.columns})
    return df

param_train = rename_columns(param_train, column_names)
param_test = rename_columns(param_test, column_names)

# loading tree encodings/representations
# encoding has the following structure: 1 value of tree height, 500 values for tip states ('1' or '2')
# 1 value for tree height and 500 values for internal node heights
# + 2 values for nb of tips of each type (to be removed) and 1 value of rescaling (removed, but stocked for rescaling predicted values back to the original scale)

encoding_train = pd.read_csv(trees_directory + 'musse.csv', sep="\t", header=None, nrows=cutoff, index_col=0, skiprows=1)
encoding_test = pd.read_csv(trees_directory + 'musse.csv', sep="\t", header=None, index_col=0, skiprows=1)

print(encoding_train.shape)
print(encoding_test.shape)



# make sure there is correspondance between indexes of dataframe with parameter values and encodings
encoding_train.index = param_train.index
encoding_test.index = param_test.index


# part of the relative path for writing down the output files
chemin = './trained_models/musse'

# the suffix of output files
expname='_236_longest_known_nb_tips_absolute_error'

# check
param_test

# check
print(encoding_test.shape)
print(param_train)

# check
encoding_test

(236, 11)
(236, 11)
(236, 502)
(236, 502)
(236, 502)
           lambda1   lambda2   lambda3       mu1       mu2       mu3  \
0                                                                      
0.581637  0.639470  0.694643  0.076380  0.339707  0.246269  0.191005   
0.171136  0.455613  0.551331  0.150403  0.076921  0.364527  0.860820   
0.587373  0.688031  0.478535  0.045761  0.677784  0.630738  0.375229   
0.267792  0.690626  0.640356  0.712357  0.443546  0.298779  0.660996   
0.454680  0.469158  0.566001  0.275795  0.631396  0.581336  0.173862   
...            ...       ...       ...       ...       ...       ...   
0.590268  0.446474  0.108840  0.420470  0.245711  0.066488  0.631408   
0.236107  0.232584  0.503103  0.162354  0.174115  0.307665  0.262108   
0.541853  0.543207  0.455826  0.161908  0.443541  0.190180  0.409562   
0.729897  0.390473  0.592883  0.200846  0.711841  0.557205  0.468335   
0.152324  0.431591  0.658767  0.241526  0.250550  0.303597  0.747699   

          

,1,2,3,4,5,6,7,8,9,10,...,493,494,495,496,497,498,499,500,501,502
0,,,,,,,,,,,,,,,,,,,,,
0.581637,7.743853,6.648208,5.721072,5.693300,6.783453,4.758588,2.604404,3.082454,7.502171,7.683798,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.672283
0.171136,5.928847,4.708325,4.516767,4.709624,5.595005,4.175818,4.114849,2.218430,2.108353,5.286968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.336636
0.587373,6.042367,1.403993,5.092303,5.867678,5.996965,0.000000,5.290625,5.884315,5.859049,6.028741,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.388216
0.267792,10.877998,10.215666,10.675867,10.160885,10.112411,10.656010,9.985865,10.306910,9.385767,9.170996,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.008681
0.454680,8.114325,0.000000,7.245628,7.855954,7.817380,6.996360,8.052058,6.958656,7.644137,7.976313,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.254778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0.590268,4.823394,1.999291,4.273489,4.169948,4.319598,1.665775,2.674232,0.161372,2.094921,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.035240
0.236107,8.959946,8.457937,7.994788,8.633631,7.806082,7.355732,8.338826,8.903481,7.082018,7.271503,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.145869
0.541853,8.076159,5.160960,6.252963,7.246771,8.001399,4.844179,7.561043,7.203715,8.044922,7.123671,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.296773


In [208]:
#correctly reshape parameters (rescaling) and encodings (remove nb of each type + rescale factor):

### TRAINING SET: PARAMETER VALUES
# rescaling factor
param_train['norm_factor'] = encoding_train[1005]
# rescale target values according to scaling factor
param_train['net_rat1_rescaled'] = param_train['net_rate1']*param_train['norm_factor']
param_train['net_rat2_rescaled'] = param_train['net_rate2']*param_train['norm_factor']
param_train['lambda1_rescaled'] = param_train['lambda1']*param_train['norm_factor']
param_train['lambda2_rescaled'] = param_train['lambda2']*param_train['norm_factor']
param_train['q01_rescaled'] = param_train['q01']*param_train['norm_factor']

### TESTING SET: PARAMETER VALUES
# rescaling factor
param_test['norm_factor'] = encoding_test[1005]
# rescale target values
param_test['net_rat1_rescaled'] = param_test['net_rate1']*param_test['norm_factor']
param_test['net_rat2_rescaled'] = param_test['net_rate2']*param_test['norm_factor']
param_test['lambda1_rescaled'] = param_test['lambda1']*param_test['norm_factor']
param_test['lambda2_rescaled'] = param_test['lambda2']*param_test['norm_factor']
param_test['q01_rescaled'] = param_test['q01']*param_test['norm_factor']

KeyError: 505

In [164]:
#correctly reshape parameters (rescaling) and encodings (remove nb of each type + rescale factor):

### TRAINING SET: PARAMETER VALUES
# norm the parameters
y_train_norm = y_train.copy()
y_train_norm['lambda1'] = y_train['lambda1'] / y_train['lambda1'].max()
y_train_norm['lambda2'] = y_train['lambda2'] / y_train['lambda2'].max()
y_train_norm['lambda3'] = y_train['lambda3'] / y_train['lambda3'].max()
y_train_norm['mu1'] = y_train['mu1'] / y_train['mu1'].max()
y_train_norm['mu2'] = y_train['mu2'] / y_train['mu2'].max()
y_train_norm['mu3'] = y_train['mu3'] / y_train['mu3'].max()
y_train_norm['q12'] = y_train['q12'] / y_train['q12'].max()
y_train_norm['q13'] = y_train['q13'] / y_train['q13'].max()
y_train_norm['q21'] = y_train['q21'] / y_train['q21'].max()
y_train_norm['q23'] = y_train['q23'] / y_train['q23'].max()
y_train_norm['q31'] = y_train['q31'] / y_train['q31'].max()
y_train_norm['q32'] = y_train['q32'] / y_train['q32'].max()

# save the normed parameters
y_train_norm.to_csv(chemin + 'y_train_norm' + expname + '.csv')

### TESTING SET: PARAMETER VALUES
# norm the parameters
y_test_norm = y_test.copy()
y_test_norm['lambda1'] = y_test['lambda1'] / y_test['lambda1'].max()
y_test_norm['lambda2'] = y_test['lambda2'] / y_test['lambda2'].max()
y_test_norm['lambda3'] = y_test['lambda3'] / y_test['lambda3'].max()
y_test_norm['mu1'] = y_test['mu1'] / y_test['mu1'].max()
y_test_norm['mu2'] = y_test['mu2'] / y_test['mu2'].max()
y_test_norm['mu3'] = y_test['mu3'] / y_test['mu3'].max()
y_test_norm['q12'] = y_test['q12'] / y_test['q12'].max()
y_test_norm['q13'] = y_test['q13'] / y_test['q13'].max()
y_test_norm['q21'] = y_test['q21'] / y_test['q21'].max()
y_test_norm['q23'] = y_test['q23'] / y_test['q23'].max()
y_test_norm['q31'] = y_test['q31'] / y_test['q31'].max()
y_test_norm['q32'] = y_test['q32'] / y_test['q32'].max()

# save the normed parameters
y_test_norm.to_csv(chemin + 'y_test_norm' + expname + '.csv')

# remove irrelevant columns: count of each type of tip and normalization factor
#### X_train = X_train.drop(columns=[1003, 1004, 1005], axis=1, inplace=True)

# save the encodings
X_train.to_csv(chemin + 'X_train' + expname + '.csv')
X_test.to_csv(chemin + 'X_test' + expname + '.csv')


In [167]:
# Read the data
X_train = pd.read_csv(chemin + 'X_train' + expname + '.csv', index_col=0)
X_test = pd.read_csv(chemin + 'X_test' + expname + '.csv', index_col=0)
y_train_norm = pd.read_csv(chemin + 'y_train_norm' + expname + '.csv', index_col=0)
y_test_norm = pd.read_csv(chemin + 'y_test_norm' + expname + '.csv', index_col=0)


#Choice of the parameters to predict

predict_all = True
if not predict_all:
    target1 = 'lambda1'
    target2 = 'lambda2'
    target3 = 'lambda3'
    target4 = 'mu1'
    target5 = 'mu2'
    target6 = 'mu3'
    target7 = 'q12'
    target8 = 'q13'
    target9 = 'q21'
    target10 = 'q23'
    target11 = 'q31'
    target12 = 'q32'
    
    targets = pd.DataFrame(y_train_norm[target1, target2, target3, target4, target5, target6, target7, target8, target9, target10, target11, target12])
    targets_test = pd.DataFrame(y_test_norm[target1, target2, target3, target4, target5, target6, target7, target8, target9, target10, target11, target12])
    
else:
    
    targets = y_train_norm
    targets_test = y_test_norm
    
features = X_train
features_test = X_test

# split in train and validation sets
X_train, X_val, y_train, y_val = train_test_split(features, targets, test_size=0.2, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)




(156, 502)
(156, 12)
(40, 502)
(40, 12)


In [168]:
    """
    #Add the known sampling fraction as 3*2 matrix into the representation (both train and test sets)
    add_target = "sampling_frac"
    added_targets = pd.DataFrame(param_train[add_target])
    features['1003'] = added_targets
    features['1004'] = added_targets
    features['1005'] = added_targets
    features['1006'] = added_targets
    features['1007'] = added_targets
    features['1008'] = added_targets

    added_targets2 = pd.DataFrame(param_test[add_target])
    features_test['1003'] = added_targets2
    features_test['1004'] = added_targets2
    features_test['1005'] = added_targets2
    features_test['1006'] = added_targets2
    features_test['1007'] = added_targets2
    features_test['1008'] = added_targets2
    """

'\n#Add the known sampling fraction as 3*2 matrix into the representation (both train and test sets)\nadd_target = "sampling_frac"\nadded_targets = pd.DataFrame(param_train[add_target])\nfeatures[\'1003\'] = added_targets\nfeatures[\'1004\'] = added_targets\nfeatures[\'1005\'] = added_targets\nfeatures[\'1006\'] = added_targets\nfeatures[\'1007\'] = added_targets\nfeatures[\'1008\'] = added_targets\n\nadded_targets2 = pd.DataFrame(param_test[add_target])\nfeatures_test[\'1003\'] = added_targets2\nfeatures_test[\'1004\'] = added_targets2\nfeatures_test[\'1005\'] = added_targets2\nfeatures_test[\'1006\'] = added_targets2\nfeatures_test[\'1007\'] = added_targets2\nfeatures_test[\'1008\'] = added_targets2\n'

In [170]:
# explore the data
print(features.shape)
print(targets.shape)
print(features_test.shape)

(196, 502)
(196, 12)
(49, 502)
